# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Write functions to parse specific information from a web page
* Iterate over successive web pages in order to create a dataset

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [48]:
from bs4 import BeautifulSoup
import requests 
import re 
import pandas as pd 

In [49]:
def mySoup(html,soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    return book_container


In [50]:
html_page = requests.get('http://books.toscrape.com/')

In [51]:
soup =BeautifulSoup(html_page.content, 'html.parser')

In [52]:
# soup.prettify

In [53]:
warning = soup.find('div', class_="alert alert-warning")
book_container = warning.nextSibling.nextSibling
# book_container

In [63]:
def retrieve_titles(book_container):
    
    titles = book_container.findAll('h3')
    
    final_titles = [h3.find('a').attrs['title'] for h3 in titles]
    
    return final_titles

In [64]:
retrieve_titles(book_container)

['A Light in the Attic',
 'Tipping the Velvet',
 'Soumission',
 'Sharp Objects',
 'Sapiens: A Brief History of Humankind',
 'The Requiem Red',
 'The Dirty Little Secrets of Getting Your Dream Job',
 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
 'The Black Maria',
 'Starving Hearts (Triangular Trade Trilogy, #1)',
 "Shakespeare's Sonnets",
 'Set Me Free',
 "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)",
 'Rip it Up and Start Again',
 'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',
 'Olio',
 'Mesaerion: The Best Science Fiction Stories 1800-1849',
 'Libertarianism for Beginners',
 "It's Only the Himalayas"]

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given html page and return a list of the star-ratings for the books. These star ratings should be formatted as integers.

In [68]:
# def retrieve_ratings(book_container):
#     regex = re.compile("star-rating (.*)")
#     book_stars = book_container.find_all('p', {"class" : regex})

#     star_ratings = []
#     for p in book_stars:
#     star_ratings.append(p.attrs['class'][-1])

#     star_dict = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
#     star_ratings = [star_dict[s] for s in star_ratings]
#     return star_ratings
    


## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [80]:
def retrieve_prices(book_container):
    prices = book_container.find_all('p', class_='price_color')
#     grab the price
    prices = [p.text for p in prices]
    prices = [float(p[1:]) for p in prices]
    return prices

In [81]:
retrieve_prices(book_container)

[51.77,
 53.74,
 50.1,
 47.82,
 54.23,
 22.65,
 33.34,
 17.93,
 22.6,
 52.15,
 13.99,
 20.66,
 17.46,
 52.29,
 35.02,
 57.25,
 23.88,
 37.59,
 51.33,
 45.17]

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [97]:
def retrieve_availabilities(book_container):
    avail = book_container('p', class_='instock availability')
    avail = [p.text.strip() for p in avail]
    return avail

In [98]:
retrieve_availabilities(book_container)

['In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock']

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [ ]:
df = pd.DataFrame()
for i in range(2,51):
    url = '"http://books.toscrape.com/catalogue/page-{}.html".format(i)'
    soup = BeautifulSoup(html_page.content, 'html.parse')
    
    
    



## Level-Up: Write a new version of the script you just wrote. 

If you used url hacking to generate each successive page url, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [ ]:
#Your code here

## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!